In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.linear_model import LinearRegression
import yaml
from assignment13 import OneDimReservoir

In [2]:
class PermeabilityEstimator(BaseEstimator, RegressorMixin, OneDimReservoir):
    
    def __init__(self, inputs, subestimator = None):
        
        #stores input dictionary as class attribute, either read from a yaml
        #file or directly from a Python dictonary
        if isinstance(inputs, str):
            with open(inputs) as f:
                self.inputs = yaml.load(f, yaml.FullLoader)
        else:
            self.inputs = inputs
        
        #Initiate OneDimReservoir and obtain initial guess
        self.reservoir = OneDimReservoir(self.inputs)
        
        #Determine size of p_plot
        self.x_grids = self.inputs['numerical']['number of grids']['x']
        self.time_steps = self.inputs['numerical']['number of time steps']
        self.frequency =  self.inputs['plots']['frequency']
        self.p_plot_size = self.time_steps * self.x_grids / self.frequency
        
        return
    
    def run_res_sim(self, k):
        #Runs through OneDimReservoir to obtain final pressure values

        #Change permeability values
        self.reservoir.k = k
        
        #Reapply initial conditions
        self.reservoir.apply_initial_conditions()
        self.reservoir.p_plot = []
        
        #Solve for pressures with new permeabilities
        self.reservoir.fill_matrices()
        
        self.reservoir.solve()
        
        self.reservoir.get_solution()
        
        self.reservoir.p_plot = np.reshape(self.reservoir.p_plot, (-1, 1))
        
        return self.reservoir.p_plot
    
    def fit(self, k, history):
        
        #Initiate OneDimReservoir function
        run_res_sim = self.run_res_sim
        
        #Create Loss Function
        loss = (lambda k:(history - run_res_sim(k).flatten())**2)
        
        #Checks if X is 2-D, y is 1-D, and standard input checks
        X, history = check_X_y(run_res_sim(k), history)
        
        #Fine real permeability
        self.coef_, junk = optimize.leastsq(loss, k)
                
        return self    
    
    def predict(self, k):
        
        #Makes sure fit has been called
        check_is_fitted(self)
        
        #Makes sure array is 2-D and standard input checks
        pressure = check_array(self.run_res_sim(k))
        
        return pressure